## Algoritmos de Clasificación (Parte I)
## Actividad 4: Modelos de clasificación
### Sebastián Contreras Zambrano

* Para poder realizar esta actividad debes haber revisado la lectura correspondiente a lasemana.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminada la actividad, comprime la carpeta y sube el .zip a la sección correspondiente.

## Sobre este ejemplo
* En esta sesión trabajaremos con una base de datos sobre clientes morosos de un banco.
Dentro de ésta se registran las siguientes observaciones:
* `default`: Variable Binaria. Registra si el cliente entró en morosidad o no.
* `income`: Ingreso promedio declarado por el cliente.
* `balance`: total del sando en la cuenta de crédito.
* `student`: Variable binaria. Registra si el cliente es estudiante o no.

## Ejercicio 1: Preparación de ambiente de trabajo
* Importe los módulos básicos para el análisis de datos.Importe las clases `LabelEncoder`, `StandardScaler` y `LabelBinarizer` de `preprocessing`
* Importe las funciones `train_test_split` y `cross_val_score` de `model_selection`
* Importe la función `classification_report` de metrics
* Importe las clases `LinearDiscriminantAnalysis` y `Quadratic DiscriminantAnalysis`.
* Agregue la base de datos en el ambiente de trabajo.
* Inspeccione la distribución de cada atributo

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder, StandardScaler, LabelBinarizer
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('default_credit.csv')
df = df.drop(columns='index')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
from sklearn.metrics import classification_report
from collections import defaultdict

def cr_to_df(y_true,y_pred, avg_measures=False):
    
    """
    Función que retorna un Dataframe de pandas a partir de un classification report de la librería sklearn, módulo metrics
    """
    
    cr = classification_report(y_true, y_pred)
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)

    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
            
    tmp_df = pd.DataFrame(D_class_data).T
    columnsTitles = ['precision', 'recall', 'f1-score', 'support']
    tmp_df = tmp_df.reindex(columns=columnsTitles)
    
    if avg_measures is False:
        tmp_df = tmp_df.drop([' micro avg', ' macro avg','weighted avg'])
    
    return tmp_df

In [ ]:
def cr_df_to_plt(df_cr):

    for index, (colname, serie) in enumerate (df_cr.iteritems()):
        plt.subplot(2,2,index+1)
        sc = sns.scatterplot(x = df_cr[colname].index, y=df_cr[colname].values, 
                         palette='dodgerblue', s=50)
        for a,b in zip(df_cr[colname].index,df_cr[colname].values):
            plt.text(a, b, str(b), fontweight ='bold')

        if colname != 'support':
            sc.set(ylim=(0, 1))
        else:
            sc.set(ylim=(0, None))
        plt.title(colname)
    plt.show()

## Ejercicio 2: Modelo base
* Recuerde que los modelos de `sklearn` no soportan datos que no sean numéricos.
* Transforme los atributos pertinentes con `LabelEncoder`.
* Genere muestras de validación y entrenamiento, reservando un 33% de los datos como validación.
* Genere un modelo con `LinearDiscriminantAnalysis` sin modificar los hiperparámetros.
* Genere métricas de evaluación utilizando `classification_report`.
* Comente sobre cuál es el desempeño del modelo en cada clase, así como en general

In [ ]:
#no recomendable para el vector objetivo!
target_label = df['default'].unique()
df['default'] = LabelEncoder().fit_transform(df['default'])

target_label = df['student'].unique()
df['student'] = LabelEncoder().fit_transform(df['student'])

In [ ]:
plt.figure(figsize=(10,8))
for index, (colname, serie) in enumerate (df.iteritems()):
    plt.subplot(2, 2, index+1)
    sns.distplot(serie, color='dodgerblue')
    plt.tight_layout()

In [ ]:
X_train_mat, X_test_mat, y_train_vec, y_test_vec = train_test_split(
    df.loc[:, 'student':'income'],
    df['default'],
    test_size=.33,
    random_state=250992)

In [ ]:
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train_mat, y_train_vec)
y_hat = lda_model.predict(X_test_mat)

In [ ]:
metricas_1 = cr_to_df(y_test_vec, y_hat)
display (metricas_1)

A partir de la tabla se pueden generar las siguientes observaciones:

* La métrica `Precision` indica que el modelo para $y_{i}=0$ tiene un __98%__ de identificaciones correctas, mientras que para $y_{i}=1$ un __75%__.


* La métrica `Recall` indica que un __26%__ de positivos reales se identificó correctamente, mientras que el __100%__ de negativos reales se identificaron de manera correcta. 


* La métrica `F1` indica que el modelo tiene un __99%__ de éxito para clasificar la categoría $y_{i}=0$ mientras que un __39%__ para clasificar a $y_{i}=1$.

In [ ]:
pd.crosstab(y_test_vec, y_hat)

* La tabla resultante permite observar las categorías predichas con las observadas. La diagonal principal reporta los casos exitosamente predichos. Una de las primeras medidas de desempeño es medir el porcentaje de casos predichos correctamente por sobre el total de casos. Esta medida se conoce como __Accuracy (Exactitud)__

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="darkgrid")
cr_df_to_plt(metricas_1)

## Ejercicio 3: Refactorización 1 - información a priori
* Dado que trabajamos con modelos generativos, podemos incluír información exógena. Para este caso agregaremos dos distribuciones:
    * Asumamos que hay un 50/50 de morosos y no morosos.
    * Asumamos que hay un 60/40 de morosos y no morosos.
* Por cada modelo, reporte las métricas de clasificación

In [ ]:
lda_model_2 = LinearDiscriminantAnalysis(priors=[.5,.5])
lda_model_2.fit(X_train_mat, y_train_vec)
y_hat_2 = lda_model_2.predict(X_test_mat)

In [ ]:
metricas_2 = cr_to_df(y_test_vec, y_hat_2)
display (metricas_2)

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="darkgrid")
cr_df_to_plt(metricas_2)

In [ ]:
lda_model_3 = LinearDiscriminantAnalysis(priors=[.6,.4])
lda_model_3.fit(X_train_mat, y_train_vec)
y_hat_3 = lda_model_3.predict(X_test_mat)

In [ ]:
metricas_3 = cr_to_df(y_test_vec, y_hat_3)
display (metricas_3)

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="darkgrid")
cr_df_to_plt(metricas_3)

### Observaciones:

Al observar los cambios realizados en los hiperparámetros de los modelos LDA, podemos apreciar que la métrica `recall` aumentó considerablemente (un __65%__ para el modelo con modificación del a prori en __50/50__ de morosos y no morosos y un __62%__ para los que a prori se asumió que __60/40__ de morosos y no morosos).

Pero esto condujo al modelo a perder hasta un __60%__ de métrica de `Precision` para la categoría $y_{i}=1$, lo cual significa que el modelo está perdiendo capacidad para identificar correctamente esta categoría.

## Ejecicio 4: Refactorización 2 - oversampling
* Uno de los problemas más graves de esta base de datos, es el fuerte desbalance entre clases. Ahora generaremos observaciones sintéticas mediante __SMOTE__ (Synthetic MinorityOversampling Technique). Para ello, debemos agregar el paquete a nuestro ambiente virtual.En nuestro terminal agregamos `conda install -c conda-forge imbalanced-learn`.Incorpore SMOTE en el ambiente de trabajo con la siguiente sintáxis `from imblearn.over_sampling import SMOTE`.
* Para implementar oversampling, debemos generar nuevos objetos que representan nuestra muestra de entrenamiento incrementada artificialmente. Para ello implemente la siguiente sintáxis:

```python

oversampler = SMOTE(random_state=11238, ratio='minority')

X_train_oversamp, y_train_oversamp = oversampler.fit_sample(X_train, y_train)
```

* Vuelva a entrenar el modelo con los datos aumentados de forma artificial y comente sobre su desempeño

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
oversampler = SMOTE(random_state=250992, ratio='minority')

In [ ]:
X_train_oversamp, y_train_oversamp = oversampler.fit_sample(X_train_mat, y_train_vec)

In [ ]:
lda_model_4 = LinearDiscriminantAnalysis()
lda_model_4.fit(X_train_oversamp, y_train_oversamp)
y_hat_4 = lda_model_4.predict(X_test_mat)

In [ ]:
metricas_4 = cr_to_df(y_test_vec, y_hat_4)
display (metricas_4)

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="darkgrid")
cr_df_to_plt(metricas_4)

Se puede observar que las métricas de `precision` y `recall` se ven empeoradas con la imputación de observaciones sintéticas mediante __SMOTE__ al dataset. 

## Ejercicio 5: Refactorización 3 - QDA
* Por último, implemente un modelo QuadraticDiscriminantAnalysis con los datos aumentados artificialmente. 
* Genere las métricas de desempeño.
* Comente a grandes rasgos sobre el mejor modelo en su capacidad predictiva.

In [ ]:
qda_model = QuadraticDiscriminantAnalysis().fit(X_train_oversamp, y_train_oversamp)
qda_class_pred = qda_model.predict(X_test_mat)

In [ ]:
metricas_qda = cr_to_df(y_test_vec, qda_class_pred)
display (metricas_qda)

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="darkgrid")
cr_df_to_plt(metricas_qda)

In [ ]:
modelos = ['LDA_no_mod_hiperparametros', 'LDA_a_prori_50/50', 'LDA_a_prori_40/60', 'LDA_con_SMOTE', 'QDA_con_SMOTE']
arr_metricas = [metricas_1, metricas_2, metricas_3, metricas_4, metricas_qda]

for i in range(len(modelos)):
    print (modelos[i])
    display(arr_metricas[i])

#### Conclusión:

Al existir un desbalanceo entre las categorías $y_{i}=1$ e $y_{i}=0$, los modelos no presentan resultados eficientes, independiente la modificación de hiperparámetros o la imputación de datos sintéticos, de hecho, al realizar modificaciones en los modelos, se puede apreciar que empeoran la capacidad para identificar correctamente los casos positivos (`presicion`).


En este mismo contexto, la métrica `Recall` mejoró significativamente, lo cual nos indica que para la categoría $y_{i}=1$, el modelo es capaz hasta en un __91%__ de de identificar correctamente los positivos reales.

El intercambio en estas métricas es signiticativo, y para elegir el _"mejor modelo"_ debemos conocer el contexto de lo que se busca obtener con la implementación de este modelo, por lo cual, el mejor modelo será el que el experto del negocio indique que es más relevante para el banco.